In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual 
from experta import *

In [2]:
ArtistD_df=pd.read_csv('./Artists In Production 1.csv')
Track_df=pd.read_csv('./Track In Production 1.csv')
genres_df=pd.read_csv('./Genre In Production.csv')

In [3]:
Track_df['rate']=0

In [4]:
pd.set_option('expand_frame_repr', False)

In [5]:
class Artist(Fact):
    pass
class Track(Fact):
    pass
class Age(Fact):
    pass

In [8]:
class MyKnowladge(KnowledgeEngine):
    
    track_df=Track_df
    x = ArtistD_df

    @Rule(NOT(Artist(W())))
    def wantArtist(self):
        self.declare(Artist(option =input('1-Choose a specific artist\n2-Skip to the track\n3-Ask about the artist then the track\n')))
    
    
    
    @Rule(Artist(option="1"))
    def askArtistName(self):
        self.name=input("What is the artist's name?\n")
        self.declare(Artist(Name=self.name))
        Track_df['rate'] = np.where(Track_df['solo'].apply(lambda x: self.name in x), Track_df['rate']+100, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
        
        
        
    @Rule(Artist(Name=W())| Artist(option ="2")|Artist(Energy=W()))
    def askActivity(self):
        dropdown = widgets.Dropdown(options = ['Partying', 'Playing Games','Playing Sport' ,'Studying',  'Working' , 'Driving','Walking' , 'Sleeping'],description='Activity:')
        display(dropdown)
        self.Activity=input("What are you doing?\n")
        dropdown.layout.display = 'none'
        self.declare(Track(Activity=self.Activity))
        Track_df['rate'] = np.where((Track_df['activity'].apply(lambda x: self.Activity in x)), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
        
        
        
    @Rule(Track(Activity=W()))
    def askmood(self):
        dropdown = widgets.Dropdown(options =    ['Happy', 'Sad', 'Stressed', 'Cheerful' ,'Angry', 'Contentment', 'Calm' ,'Romantic'],description='Mood:')
        display(dropdown)
        self.Mood=input("How are you feeling?\n")
        dropdown.layout.display = 'none'
        self.declare(Track(Mood=self.Mood))
        Track_df['rate'] = np.where(Track_df['mood'].apply(lambda x: self.Mood in x), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
 


    @Rule(Track(Mood=W()))
    def askFavsong(self):
        self.Favsong=input("Do you have a favourite song, if so what is it?\n(no , name of the song)\n")
        self.declare(Track(Favsong=self.Favsong))
        self.s=100
        if self.Favsong=='no':
                return
        try:
            self.s=int(Track_df[['Track_name','cluster_song_label']].loc[Track_df['Track_name']==self.Favsong].cluster_song_label.values[0])
        except:
            print("\n Couldnt find the song \n")
        self.declare(Track(Favsong=self.s))
        Track_df['rate'] = np.where((Track_df['cluster_song_label'] ==self.s), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df.sort_values(["rate"], ascending= False).head(10))
        
        
        
    @Rule(Track(Favsong=W()))
    def askGenre(self):
        dropdown = widgets.Dropdown(options =sorted(genres_df.genres.unique()))
        display(dropdown)
        self.genre=input("What is your favourite genre?\n")
        dropdown.layout.display = 'none'
        self.declare(Track(Genre=self.genre))
        Track_df['rate'] = np.where(Track_df['genres'].apply(lambda x: self.genre in x), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
    
    
    
    @Rule(Track(Genre=W()))
    def askInstrument(self):
        self.instrument=input("Do you like instrument sounds?\n(yes , no)\n")
        self.declare(Track(Instrument=self.instrument))
        if self.instrument=='no':
            Track_df['rate'] = np.where((Track_df['Track_instrumentalness_perc'].apply(lambda x: 'Very Low' in x))
                                            |(Track_df['Track_instrumentalness_perc'].apply(lambda x: 'Low' in x))
                                            |(Track_df['Track_instrumentalness_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        if self.instrument=='yes':
            Track_df['rate'] = np.where((Track_df['Track_instrumentalness_perc'].apply(lambda x: 'Very High' in x))
                                            |(Track_df['Track_instrumentalness_perc'].apply(lambda x: 'High' in x))
                                            |(Track_df['Track_instrumentalness_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))    
    
    
    
    @Rule(Track(Instrument=W()))
    def askAcousticness(self):
        self.acous=input("Do you enjoy acoustic songs?\n(yes , no)\n")
        self.declare(Track(Acousticness=self.acous))
        if self.acous=='no':
                Track_df['rate'] = np.where((Track_df['Track_acousticness_perc'].apply(lambda x: 'Very Low' in x))
                                            |(Track_df['Track_acousticness_perc'].apply(lambda x: 'Low' in x))
                                            |(Track_df['Track_acousticness_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))

        
        
    
    @Rule(Track(Acousticness=W()))
    def askPopularity(self):
        self.popularity=input("Do you want popular songs?\n(yes , no)\n")
        self.declare(Track(Popularity=self.popularity))
        if self.popularity=='yes':
            Track_df['rate'] = np.where((Track_df['Popularity_perc'].apply(lambda x: 'Very High' in x))
                                            |(Track_df['Popularity_perc'].apply(lambda x: 'High' in x)), Track_df['rate']+1, Track_df['rate'])
        if self.popularity=='no':
            Track_df['rate'] = np.where((Track_df['Popularity_perc'].apply(lambda x: 'Very Low' in x))
                                            |(Track_df['Popularity_perc'].apply(lambda x: 'Low' in x))
                                            |(Track_df['Popularity_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
        
    
    
    @Rule(Track(Popularity=W()))
    def askReleaseDate(self):
        self.release=input("Do you prefer old or new songs?\n(old , new)\n")
        self.declare(Track(ReleaseDate=self.release))
        if self.release=='new':
            Track_df['rate'] = np.where(Track_df.year > 2010, Track_df['rate']+1, Track_df['rate'])
        if self.release=='old':
            Track_df['rate'] = np.where(Track_df.year < 2010, Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate','explicit']].sort_values(["rate"], ascending= False).head(20))
    
    
    
    @Rule(Track(ReleaseDate=W()))
    def askAge(self):
        self.age=input("How old are you?\n")
        self.declare(Track(Explicit=self.age))
        
        if int(self.age)<13 :
            Track_df['rate'] = np.where(Track_df.explicit ==1, Track_df['rate']-10, Track_df['rate'])
        print(Track_df[['Track_name','solo','rate',"Popularity_perc"]].sort_values(["rate","Popularity_perc"], ascending= False).head(20))

        

    @Rule(Artist(option ="3"))
    def askForFavArtist(self):
        self.FavArtist=input("Who is your favourite artist?\n")
        self.s=100
        try:
            self.s=int(ArtistD_df[['artists','cluster_artists_label']].loc[ArtistD_df['artists']==self.FavArtist].cluster_artists_label.values[0])
        except:
            print("\n Couldnt find the artist \n")
        self.declare(Artist(Fav=self.s))
        Track_df['rate'] = np.where(Track_df['cluster_song_label']==self.s, Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
    
    
    
    @Rule(Artist(Fav=W()))
    def askPopularityForArtist(self):
        self.popularity=input("Do you want popular artists?\n(yes , no)\n")
        self.declare(Artist(Popularity=self.popularity))
        if self.popularity=='yes':
            Track_df['rate'] = np.where((Track_df['popularity_perc'].apply(lambda x: 'Very High' in x))
                                            |(Track_df['popularity_perc'].apply(lambda x: 'High' in x))
                                            |(Track_df['popularity_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        if self.popularity=='no':
            Track_df['rate'] = np.where((Track_df['popularity_perc'].apply(lambda x: 'Very Low' in x))
                                            |(Track_df['popularity_perc'].apply(lambda x: 'Low' in x))
                                            |(Track_df['popularity_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
    
    
    
    @Rule(Artist(Popularity=W()))
    def askGenreForArtist(self):
        dropdown = widgets.Dropdown(options =    sorted(genres_df.genres.unique()))
        display(dropdown)
        self.genreArtist=input("What genre do u prefer the artist to be?\n")
        dropdown.layout.display = 'none'
        self.declare(Artist(Genre=self.genreArtist))
        Track_df['rate'] = np.where(Track_df['genres'].apply(lambda x: self.genreArtist in x), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))
    
    
    
    @Rule(Artist(Genre=W()))
    def askEnergeticForArtist(self):
        self.energyArtist=input("Do you prefer energetic artists?\n(yes , no)\n")
        self.declare(Artist(Energy=self.energyArtist))
        if self.energyArtist=='yes':
            Track_df['rate'] = np.where((Track_df['energy_perc'].apply(lambda x: 'Very High' in x))
                                            |(Track_df['energy_perc'].apply(lambda x: 'High' in x))
                                            |(Track_df['energy_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        if self.energyArtist=='no':
            Track_df['rate'] = np.where((Track_df['energy_perc'].apply(lambda x: 'Very Low' in x))
                                            |(Track_df['energy_perc'].apply(lambda x: 'Low' in x))
                                            |(Track_df['energy_perc'].apply(lambda x: 'Average' in x)), Track_df['rate']+1, Track_df['rate'])
        #print(Track_df[['Track_name','solo','rate']].sort_values(["rate"], ascending= False).head(20))   

In [9]:
Track_df['rate']=0
engine = MyKnowladge()
engine.reset()
engine.run()

1-Choose a specific artist
2-Skip to the track
3-Ask about the artist then the track
2


Dropdown(description='Activity:', options=('Partying', 'Playing Games', 'Playing Sport', 'Studying', 'Working'…

What are you doing?
w


Dropdown(description='Mood:', options=('Happy', 'Sad', 'Stressed', 'Cheerful', 'Angry', 'Contentment', 'Calm',…

How are you feeling?
w
Do you have a favourite song, if so what is it?
(no , name of the song)
w

 Couldnt find the song 



Dropdown(options=('21st century classical', '432hz', '8-bit', '[]', 'a cappella', 'abstract', 'abstract beats'…

What is your favourite genre?
w
Do you like instrument sounds?
(yes , no)
yes
Do you enjoy acoustic songs?
(yes , no)
yes
Do you want popular songs?
(yes , no)
yes
Do you prefer old or new songs?
(old , new)
new
How old are you?
15


KeyError: 'rate'